# **기사 전처리(ngramize까지)**

In [1]:
# import
import re
import csv
import numpy as np
import pandas as pd
import MeCab
from ekonlpy.sentiment import MPCK
from ekonlpy.tag import Mecab

In [2]:
# 기간별 기사가 저장된 CSV파일들을 불러옴
arti_df1 = pd.read_csv('article_2005_20110706.csv')
arti_df2 = pd.read_csv('article_20110707_20160202.csv')
arti_df3 = pd.read_csv('article_20160203_2017.csv')

## **기간별 기사들을 하나의 DataFrame으로 만듬.**

In [3]:
# 각 기사들 중 연합뉴스의 기사들을 각 변수에 저장
yonhap1 = arti_df1[arti_df1['logo'] == '연합뉴스']
yonhap2 = arti_df2[arti_df2['logo'] == '연합뉴스']
yonhap3 = arti_df3[arti_df3['logo'] == '연합뉴스']

In [4]:
# 기간별 기사들을 합친 후 인덱스 재설정
frames = [yonhap1, yonhap2, yonhap3]
yonhap_df = pd.concat(frames)
yonhap_df.reset_index(drop=True, inplace=True)
yonhap_df

,date,logo,text,title,url
0,2005.01.01. 오전 7:01,연합뉴스,(단위:%) ...,<표> 은행 금전신탁배당률.정기예금 금리(12.31일),https://news.naver.com/main/read.nhn?mode=LSD&...
1,2005.01.17. 오후 12:02,연합뉴스,농민 자녀 장학기금 2012년까지 1천억원으로 확대\r\n (서울=연합뉴스) ...,농협 상호금융 대출금리 연 5%대 초반으로 인하,https://news.naver.com/main/read.nhn?mode=LSD&...
2,2005.01.12. 오전 6:06,연합뉴스,(서울=연합뉴스) 김문성기자= 증권사들이 개인 투자자를 증시로 끌어들이기 위해 위...,`개인 투자자 잡아라'..증거금.대출금리 인하,https://news.naver.com/main/read.nhn?mode=LSD&...
3,2005.01.12. 오전 11:45,연합뉴스,(서울=연합뉴스) 신호경기자= 씨티그룹글로벌마켓증권(이하 씨티그룹증권)은 오는 13...,"""이달 금리동결 전망..2월 인하 가능성""<씨티그룹증권>",https://news.naver.com/main/read.nhn?mode=LSD&...
4,2005.01.12. 오후 3:45,연합뉴스,(서울=연합뉴스) 신호경기자= 모건스탠리는 12일 올해 중국이 점진적 금리 인상에 ...,"""中, 올해 점진적 금리인상이 최선""<모건스탠리>",https://news.naver.com/main/read.nhn?mode=LSD&...
...,...,...,...,...,...
70163,2017.12.30. 오전 11:05,연합뉴스,(상하이=연합뉴스) 정주호 특파원 = 올해 들어 중국 위안화 가치가 5.8% 오르며...,中 위안화 가치 1년새 5.8% 상승…9년만에 최대폭,https://news.naver.com/main/read.nhn?mode=LSD&...
70164,2017.12.30. 오전 8:03,연합뉴스,"국내 주식형, 패시브펀드가 액티브펀드 상회 (서울=연합뉴스) 윤선희 기자 = 전 세...",중국·베트남펀드 올해 수익률 30%대로 최고,https://news.naver.com/main/read.nhn?mode=LSD&...
70165,2017.12.29. 오전 10:05,연합뉴스,(서울=연합뉴스) 이춘규 기자 = 미국과 유럽이 세계 금융위기 뒤 지속한 금융완화에...,"""마이너스금리 일본은행서 드디어 금리인상 검토 목소리""",https://news.naver.com/main/read.nhn?mode=LSD&...
70166,2017.12.29. 오후 12:00,연합뉴스,"고객 몰래 대출금리 인상 혐의…법원 ""무단으로 인상했다 단정 못해""(서울=연합뉴스)...",'대출금리 조작 사건' 외환은행 전·현직 임원 무죄 확정,https://news.naver.com/main/read.nhn?mode=LSD&...


In [5]:
# 각 기사들 중 이데일리의 기사들을 각 변수에 저장
eday1 = arti_df1[arti_df1['logo'] == '이데일리']
eday2 = arti_df2[arti_df2['logo'] == '이데일리']
eday3 = arti_df3[arti_df3['logo'] == '이데일리']

In [6]:
# 기간별 기사들을 합친 후 인덱스 재설정
eday_frames = [eday1, eday2, eday3]
eday_df = pd.concat(eday_frames)
eday_df.reset_index(drop=True, inplace=True)
eday_df

,date,logo,text,title,url
0,2005.01.12. 오후 12:01,이데일리,[edaily 박기수기자] 작년 한해동안 주택금융공사의 모기지론(장기주택담보대출)을...,"작년 고정금리 모기지론 4만7천여명, 3.3조원 대출받아",https://news.naver.com/main/read.nhn?mode=LSD&...
1,2005.01.12. 오후 1:49,이데일리,[edaily 이정훈기자] 최근 장기금리를 중심으로 한 시중금리 상승에 대해 KB자...,"KB자산운용 백경호 사장 ""現 금리상승 일시적""",https://news.naver.com/main/read.nhn?mode=LSD&...
2,2005.01.18. 오후 10:26,이데일리,"[뉴욕=edaily 정명수특파원] 안소니 산토메로 필라델피아 연방은행 총재는 ""올해...","""금리인상 점진적""-필라델피아FRB총재",https://news.naver.com/main/read.nhn?mode=LSD&...
3,2005.01.01. 오전 7:30,이데일리,"[뉴욕=edaily 안근모특파원] 2004년 마지막 거래일의 부진이 아쉽긴 하지만,...",(월가시각)내일에는 내일의 태양이,https://news.naver.com/main/read.nhn?mode=LSD&...
4,2005.01.12. 오전 8:38,이데일리,[edaily 강종구기자]시티글로벌마켓(CGM)은 12일 한국은행의 콜금리 인하에 ...,금리인하는 1월 아닌 2월-시티(1보),https://news.naver.com/main/read.nhn?mode=LSD&...
...,...,...,...,...,...
99858,2017.12.30. 오전 8:01,이데일리,[이데일리 권소현 기자] ‘부익부 빈인빈’내년 주택 시장은 전반적으로 약보합을 이어...,[2018 부동산 전망]①규제에 입주폭탄까지…보합 속 양극화,https://news.naver.com/main/read.nhn?mode=LSD&...
99859,2017.12.29. 오후 3:59,이데일리,[이데일리 김정현 기자] 가계부채 비중이 높은 우리나라에서 금리정책은 경기부양보다는...,"""韓, 가계부채 높아 금리 인상시 소비에 부정적""",https://news.naver.com/main/read.nhn?mode=LSD&...
99860,2017.12.30. 오전 6:02,이데일리,[이데일리 박종오 유현욱 기자] 올해 금융권 주요 이슈를 한 단어로 설명할 수 있는...,인터넷뱅크·비트코인 열풍…금융권 올해 키워드는 ‘급변’,https://news.naver.com/main/read.nhn?mode=LSD&...
99861,2017.12.30. 오전 8:01,이데일리,[이데일리 권소현 기자] 내년 부동산 시장에 관망세가 짙을 것으로 예상되는 가운데 ...,[2018 부동산 전망]③양도세 중과 직전이 내집마련 적기,https://news.naver.com/main/read.nhn?mode=LSD&...


In [23]:
# 각 기사들 중 연합인포맥스의 기사들을 각 변수에 저장
yonhap_in1 = arti_df1[arti_df1['logo'] == '연합인포맥스']
yonhap_in2 = arti_df2[arti_df2['logo'] == '연합인포맥스']
yonhap_in3 = arti_df3[arti_df3['logo'] == '연합인포맥스']

In [24]:
# 기간별 기사들을 합친 후 인덱스 재설정
yonhap_in_frames = [yonhap_in1, yonhap_in2, yonhap_in3]
yonhap_in_df = pd.concat(yonhap_in_frames)
yonhap_in_df.reset_index(drop=True, inplace=True)
yonhap_in_df

,date,logo,text,title,url
0,2005.01.18. 오후 2:12,연합인포맥스,(서울=연합인포맥스) 이현중기자= 18일 오후 채권시장은 심리회복에 따른 매수세가...,"채권금리, 심리회복..본격 매수세 회복은 아직",https://news.naver.com/main/read.nhn?mode=LSD&...
1,2005.01.12. 오후 10:54,연합인포맥스,(뉴욕=연합인포맥스) 김홍규 특파원= 캐서린 미네한 보스턴연방준비은행 총재는12일 ...,FRB 기준금리 추가 인상 필요[보스턴연銀 총재],https://news.naver.com/main/read.nhn?mode=LSD&...
2,2005.01.12. 오전 10:15,연합인포맥스,"(서울=연합인포맥스) 이규창기자= 향후 新BIS 협약인 바젤Ⅱ가 발효되면, 우리나...","바젤Ⅱ 발효시, 외평채 가산금리 상승",https://news.naver.com/main/read.nhn?mode=LSD&...
3,2005.01.18. 오전 10:44,연합인포맥스,(서울=연합인포맥스) 이한용 기자= 18일 아시아에서 스크린을 통해 거래되는 미국채...,"美 국채가, FRB 금리인상 시사 관측 亞서 하락 전망",https://news.naver.com/main/read.nhn?mode=LSD&...
4,2005.01.18. 오전 9:16,연합인포맥스,(서울=연합인포맥스) 이현중기자= 18일 오전 채권시장은 올들어 이어진 금리 급등흐...,"채권금리, 내림세 출발..심리 안정 기대",https://news.naver.com/main/read.nhn?mode=LSD&...
...,...,...,...,...,...
80917,2013.01.02. 오전 8:51,연합인포맥스,(서울=연합인포맥스) 권용욱 기자 = 2일 국채선물시장은 미국 재정절벽 타결 협상에...,[국채선물 전략] 미국發 약세 압력,https://news.naver.com/main/read.nhn?mode=LSD&...
80918,2013.01.02. 오전 8:33,연합인포맥스,"(서울=연합인포맥스) 정지서 기자 = 박재식 한국증권금융 사장은 2일 ""실적 악화가...","박재식 한국증금 사장 ""실적악화 불가피, 중장기 로드맵 필수""",https://news.naver.com/main/read.nhn?mode=LSD&...
80919,2013.01.02. 오전 9:00,연합인포맥스,(서울=연합인포맥스) 권용욱 기자 = 2일 단기자금시장은 통안채 만기 등으로 지급준...,[단기자금시장 분석] 통안채 만기..지준 잉여,https://news.naver.com/main/read.nhn?mode=LSD&...
80920,2013.01.02. 오전 8:06,연합인포맥스,(서울=연합인포맥스) 2일 서울채권시장은 미국의 재정절벽 협상이 연말 극적으로타결된...,[한창헌의 채권분석] 재정절벽 타결에 되돌림 압력,https://news.naver.com/main/read.nhn?mode=LSD&...


## **전처리 함수**

In [12]:
def multi_pre_processing(df, doc_name_str, max_iterate) : 
    test_list = [[1,2,3,4,5,6]]  # 데이터 프레임을 만들기 위한 test_list
    df_pre_processing = pd.DataFrame(test_list,columns =['date', 'title', 'content','tokens', 'ngrams', 'callLabel'])
    df_pre_processing = df_pre_processing.drop([0])  # test_list 제거
    
    mecab = Mecab()
    mpck = MPCK()
    
    for i in range(max_iterate) :
        if i%100 == 0:
            print('{}/{}'.format(i, max_iterate))
        
        text = str(df['text'][i])
        text = str(df['text'][i]).replace('\n', '')  # 줄바꿈 제거

        # 이데일리 전처리
        text2 = re.sub('\[.*\]', '', text)  # [서울=edaily 배진우 기자] 등 제거
        text3 = re.sub('Copyright.+\(?[a-zA-z0-9]+@[a-zA-z.]+\)?', '', text2)  # Copyright ... 배진우 기자 (bigboom@edaily.co.kr) 제거
        
        # 연합뉴스, 연합인포맥스 전처리
        text4 = re.sub('\(\w*=\w*\)', '', text3)  # (부산=연합뉴스), (서울=연합인포맥스) 등 제거
        text5 = re.sub('[\w\s]+기자\s?=', '', text4)  # 배진우 기자 =, 배진우 기자= 제거
        text6 = re.sub('[\w\s]+특파원\s?=', '', text5)  # 하은혜 특파원 =, 조하담 특파원= 제거
        text7 = re.sub('[a-zA-z0-9]+@[a-zA-z.]+', '', text6)  # goisugn@yna.co.kr 제거
        text8 = re.sub('\(끝\)', '', text7).strip()  # (끝) 제거
        
        date = str(df['date'][i])[:10].replace('.','-')  # 날짜에서 '.'을 '-'로 변경
        title = str(df['title'][i])
        
        tokens = mpck.tokenize(text8)
        
        ngrams = mpck.ngramize(tokens)
        
        # ngrams가 없을 경우 저장하지 않음
        if not ngrams:
            continue
        
        df_pre_processing.loc[i] = [date, title, text8, tokens, ngrams, 0]

    df_pre_processing.reset_index(drop=True, inplace=True)  # 인덱스 재 설정 
    df_pre_processing.to_csv('{}.csv'.format(doc_name_str))

## **TEST**

In [13]:
multi_pre_processing(yonhap_df,'yonhap',len(yonhap_df))

0/70168
100/70168
200/70168
300/70168
400/70168
500/70168
600/70168
700/70168
800/70168
900/70168
1000/70168
1100/70168
1200/70168
1300/70168
1400/70168
1500/70168
1600/70168
1700/70168
1800/70168
1900/70168
2000/70168
2100/70168
2200/70168
2300/70168
2400/70168
2500/70168
2600/70168
2700/70168
2800/70168
2900/70168
3000/70168
3100/70168
3200/70168
3300/70168
3400/70168
3500/70168
3600/70168
3700/70168
3800/70168
3900/70168
4000/70168
4100/70168
4200/70168
4300/70168
4400/70168
4500/70168
4600/70168
4700/70168
4800/70168
4900/70168
5000/70168
5100/70168
5200/70168
5300/70168
5400/70168
5500/70168
5600/70168
5700/70168
5800/70168
5900/70168
6000/70168
6100/70168
6200/70168
6300/70168
6400/70168
6500/70168
6600/70168
6700/70168
6800/70168
6900/70168
7000/70168
7100/70168
7200/70168
7300/70168
7400/70168
7500/70168
7600/70168
7700/70168
7800/70168
7900/70168
8000/70168
8100/70168
8200/70168
8300/70168
8400/70168
8500/70168
8600/70168
8700/70168
8800/70168
8900/70168
9000/70168
9100/70168


In [14]:
multi_pre_processing(eday_df,'eday',100)

0/100
1/100
2/100
3/100
4/100
5/100
6/100
7/100
8/100
9/100
10/100
11/100
12/100
13/100
14/100
15/100
16/100
17/100
18/100
19/100
20/100
21/100
22/100
23/100
24/100
25/100
26/100
27/100
28/100
29/100
30/100
31/100
32/100
33/100
34/100
35/100
36/100
37/100
38/100
39/100
40/100
41/100
42/100
43/100
44/100
45/100
46/100
47/100
48/100
49/100
50/100
51/100
52/100
53/100
54/100
55/100
56/100
57/100
58/100
59/100
60/100
61/100
62/100
63/100
64/100
65/100
66/100
67/100
68/100
69/100
70/100
71/100
72/100
73/100
74/100
75/100
76/100
77/100
78/100
79/100
80/100
81/100
82/100
83/100
84/100
85/100
86/100
87/100
88/100
89/100
90/100
91/100
92/100
93/100
94/100
95/100
96/100
97/100
98/100
99/100
100/100


In [25]:
multi_pre_processing(yonhap_in_df,'yonhap_in',len(yonhap_in_df))

0/80922
100/80922
200/80922
300/80922
400/80922
500/80922
600/80922
700/80922
800/80922
900/80922
1000/80922
1100/80922
1200/80922
1300/80922
1400/80922
1500/80922
1600/80922
1700/80922
1800/80922
1900/80922
2000/80922
2100/80922
2200/80922
2300/80922
2400/80922
2500/80922
2600/80922
2700/80922
2800/80922
2900/80922
3000/80922
3100/80922
3200/80922
3300/80922
3400/80922
3500/80922
3600/80922
3700/80922
3800/80922
3900/80922
4000/80922
4100/80922
4200/80922
4300/80922
4400/80922
4500/80922
4600/80922
4700/80922
4800/80922
4900/80922
5000/80922
5100/80922
5200/80922
5300/80922
5400/80922
5500/80922
5600/80922
5700/80922
5800/80922
5900/80922
6000/80922
6100/80922
6200/80922
6300/80922
6400/80922
6500/80922
6600/80922
6700/80922
6800/80922
6900/80922
7000/80922
7100/80922
7200/80922
7300/80922
7400/80922
7500/80922
7600/80922
7700/80922
7800/80922
7900/80922
8000/80922
8100/80922
8200/80922
8300/80922
8400/80922
8500/80922
8600/80922
8700/80922
8800/80922
8900/80922
9000/80922
9100/80922


In [14]:
df_yonhap_test = pd.read_csv('yonhap.csv', index_col=0)
df_yonhap_test

,date,title,content,tokens,ngrams,callLabel
0,2005-01-17,농협 상호금융 대출금리 연 5%대 초반으로 인하,현행 연 6%대 후반인 농협 상호금융 대출금리가 올해안에 연 5%대 초반까지 단계...,"['연/NNG', '농협/NNG', '상호금융/NNG', '대출/NNG', '금리/...","['농가/NNG;소득/NNG;안정/NNG', '비용/NNG;부담/NNG;덜/VV',...",0
1,2005-01-12,`개인 투자자 잡아라'..증거금.대출금리 인하,증권사들이 개인 투자자를 증시로 끌어들이기 위해 위탁 증거금 적용 비율과 대출 금...,"['개인/NNG', '증시/NNG', '끌/VV', '위하/VV', '위탁증거금/N...","['대출/NNG;금리/NNG;인하/NNG', '대출/NNG;금리/NNG;하향/NNG...",0
2,2005-01-12,"""이달 금리동결 전망..2월 인하 가능성""<씨티그룹증권>",씨티그룹글로벌마켓증권(이하 씨티그룹증권)은 오는 13일 열리는 금융통화위원회에서 한...,"['증권/NNG', '오/VV', '열/VV', '콜/NNG', '금리/NNG', ...","['국고채/NNG;물량/NNG;부담/NNG', '채권시장/NNG;불안/NNG', '...",0
3,2005-01-12,"""中, 올해 점진적 금리인상이 최선""<모건스탠리>",모건스탠리는 12일 올해 중국이 점진적 금리 인상에 나서는 한편 통화가치는 현 수준...,"['점진적/VAX', '금리/NNG', '인상/NNG', '나서/VV', '통화/N...","['인플레이션/NNG;위험/NNG;높/VA;금리/NNG;인상/NNG', '금리/NN...",0
4,2005-01-18,"한국타이어, 지난해 사상최대 실적 달성(종합)",한국타이어가 수출 호조와 고부가가치 제품 판매 증가에 힘입어 지난해 사상 최대 실적...,"['수출/NNG', '호조/NNG', '고부가가치/NNG', '제품/NNG', '판...","['영업이익/NNG;순이익/NNG;증가/NNG', '제품/NNG;판매/NNG;증가/...",0
...,...,...,...,...,...,...
61316,2017-12-30,中 위안화 가치 1년새 5.8% 상승…9년만에 최대폭,올해 들어 중국 위안화 가치가 5.8% 오르며 9년만에 최대폭 상승을 기록했다. ...,"['들/VV', '위안/NNG', '오르/VV', '대폭/NNG', '상승/NNG'...","['통화정책/NNG;완화/NNG', '하락/NNG;제한적/VAX', '절하/NNG;...",0
61317,2017-12-30,중국·베트남펀드 올해 수익률 30%대로 최고,"국내 주식형, 전 세계 증시가 동반 강세를 펼친 올해 펀드시장에서 중국펀드와 베트남...","['주식형/NNG', '전세계/NNG', '증시/NNG', '동반/NNG', '강세...","['강하/VA;금리/NNG;상승/NNG', '상승률/NNG;넘/VV', '심리/NN...",0
61318,2017-12-29,"""마이너스금리 일본은행서 드디어 금리인상 검토 목소리""",미국과 유럽이 세계 금융위기 뒤 지속한 금융완화에서 벗어나려는 가운데 일본은행 내에...,"['세계/NNG', '금융위기/NNG', '뒤/NNG', '지속/NNG', '금융/...","['정책/NNG;효과/NNG;부작용/NNG', '정책/NNG;금융/NNG;완화/NN...",0
61319,2017-12-29,'대출금리 조작 사건' 외환은행 전·현직 임원 무죄 확정,"고객 몰래 대출금리 인상 혐의…법원 ""무단으로 인상했다 단정 못해"" 고객 동의 없이...","['고객/NNG', '몰래/MAG', '대출/NNG', '금리/NNG', '인상/N...","['대출/NNG;스프레드/NNG;인상/NNG', '대출/NNG;금리/NNG;인상/N...",0


In [16]:
df_eday_test = pd.read_csv('eday.csv', index_col=0)
df_eday_test

,date,title,content,tokens,ngrams,callLabel
0,2005-01-12,"작년 고정금리 모기지론 4만7천여명, 3.3조원 대출받아",작년 한해동안 주택금융공사의 모기지론(장기주택담보대출)을 통해 4만7841명이 3조...,"['모기지/NNG', '장기/NNG', '주택담보대출/NNG', '통하/VV', '...",['실적/NNG;오르/VV'],0
1,2005-01-12,"KB자산운용 백경호 사장 ""現 금리상승 일시적""","최근 장기금리를 중심으로 한 시중금리 상승에 대해 KB자산운용 백경호 사장은 ""마찰...","['장기/NNG', '금리/NNG', '중심/NNG', '하/VV', '상승/NNG...","['개선/NNG;내수/NNG;회복/NNG', '내수/NNG;회복/NNG;지연/NNG...",0
2,2005-01-18,"""금리인상 점진적""-필라델피아FRB총재","안소니 산토메로 필라델피아 연방은행 총재는 ""올해 경제는 좀 더 탄탄한 성장세를 나...","['경제/NNG', '좀/MAG', '더/MAG', '탄탄/NNG', '성장/NNG...","['생산성/NNG;증가율/NNG;둔화/NNG', 'fed/NNG;금리/NNG;인상/...",0
3,2005-01-01,(월가시각)내일에는 내일의 태양이,"2004년 마지막 거래일의 부진이 아쉽긴 하지만, 할 만큼은 다 했다. 라이언 벡 ...","['마지막/NNG', '거래일/NNG', '부진/NNG', '아쉽/VA', '하/V...","['걸림돌/NNG;금리/NNG;인상/NNG', '경제/NNG;성장률/NNG;낮/VA...",0
4,2005-01-12,금리인하는 1월 아닌 2월-시티(1보),시티글로벌마켓(CGM)은 12일 한국은행의 콜금리 인하에 대한 기대가 높지만 그 시...,"['시티/NNG', '글로벌/NNG', '시장/NNG', '콜/NNG', '금리/N...","['콜/NNG;금리/NNG;인하/NNG;기대/NNG;높/VA', '세계/NNG;경제...",0
...,...,...,...,...,...,...
81,2005-01-20,(고령화시대)③채권시장이 변한다,저출산·고령화는 채권시장에도 직간접적인 영향을 미친다. 채권시장은 고령화와 함께 저...,"['고령화/NNG', '채권시장/NNG', '직간접적/NNG', '영향/NNG', ...","['채권시장/NNG;콜/NNG;금리/NNG;인하/NNG', '장기물/NNG;시장/N...",0
82,2005-01-20,(마켓서핑)싹트는 불안감,채권시장은 물량부담에 대한 불안감이 투자심리를 위축시켰고 주식시장에서는 단기 급등에...,"['채권시장/NNG', '물량/NNG', '부담/NNG', '불안/NNG', '투자...","['단기/NNG;급등/NNG;가격/NNG;부담/NNG', '채권시장/NNG;물량/N...",0
83,2005-01-20,"""실적 들쭉날쭉"" 나스닥 2100 무너져",다우와 나스닥이 약보합선에 머물러 있다. 나스닥은 다시 2100선이 무너졌다. 기업...,"['다우/NNG', '나스닥/NNG', '약보합선/NNG', '머무르/VV', '나...","['근원/NNG;소비자/NNG;물가/NNG;상승/NNG', '경제/NNG;지표/NN...",0
84,2005-01-20,"""실적 신통치 않다""..나스닥 1.5%↓",다우와 나스닥이 큰 폭으로 떨어졌다. 기술주 실적에 대한 의구심이 고조되면서 장막판...,"['다우/NNG', '나스닥/NNG', '크/VA', '떨어/VV', '실적/NNG...","['국채/NNG;수익률/NNG;소폭/MAG;하락/NNG', '근원/NNG;소비자/N...",0


In [26]:
df_yonhap_in_test = pd.read_csv('yonhap_in.csv', index_col=0)
df_yonhap_in_test

,date,title,content,tokens,ngrams,callLabel
0,2005-01-18,"채권금리, 심리회복..본격 매수세 회복은 아직",18일 오후 채권시장은 심리회복에 따른 매수세가 유입되면서 주요 기물별 금리하락폭...,"['채권시장/NNG', '심리/NNG', '회복/NNG', '따르/VV', '매수/...","['채권시장/NNG;심리/NNG;회복/NNG', '금리/NNG;하락/NNG;크/VV...",0
1,2005-01-12,FRB 기준금리 추가 인상 필요[보스턴연銀 총재],캐서린 미네한 보스턴연방준비은행 총재는12일 연방준비제도이사회(FRB)가 여전히 기...,"['보스턴/NNG', 'fed/NNG', 'fed/NNG', '여전히/MAG', '...","['경상/NNG;적자/NNG', '완만/NNG;성장/NNG']",0
2,2005-01-12,"바젤Ⅱ 발효시, 외평채 가산금리 상승","향후 新BIS 협약인 바젤Ⅱ가 발효되면, 우리나라의 외국환평형기금채권(외평채)의 ...","['新/NNG', 'bis/NNG', '협약/NNG', '바젤/NNG', '발효/N...","['외평채/NNG;스프레드/NNG;상승/NNG', '채권/NNG;스프레드/NNG;상...",0
3,2005-01-18,"美 국채가, FRB 금리인상 시사 관측 亞서 하락 전망",18일 아시아에서 스크린을 통해 거래되는 미국채 가격은 연방준비제도이사회(FRB) ...,"['스크린/NNG', '통하/VV', '거래/NNG', '되/XSV', '가격/NN...","['인상/NNG;관측/NNG;강화/NNG', '경제/NNG;회복/NNG;가속/NNG...",0
4,2005-01-18,"채권금리, 내림세 출발..심리 안정 기대",18일 오전 채권시장은 올들어 이어진 금리 급등흐름이 주춤하면서 내림세로 출발했다....,"['채권시장/NNG', '들/VV', '이어지/VV', '금리/NNG', '급등/N...","['금리/NNG;급등/NNG', '심리/NNG;안정/NNG', '기대/NNG;살/VV']",0
...,...,...,...,...,...,...
73973,2013-01-02,[국채선물 전략] 미국發 약세 압력,2일 국채선물시장은 미국 재정절벽 타결 협상에 대한 기대 등으로 약세 압력을 받을 ...,"['국채선물시장/NNG', '재정절벽/NNG', '타결/NNG', '협상/NNG',...","['국채선물/NNG;가격/NNG;하락/NNG', '안전자산/NNG;선호/NNG;약화...",0
73974,2013-01-02,"박재식 한국증금 사장 ""실적악화 불가피, 중장기 로드맵 필수""","박재식 한국증권금융 사장은 2일 ""실적 악화가 불가피한 올해엔 중장기 성장 동력을 ...","['실적/NNG', '악화/NNG', '불가피/NNG', '엔/NNG', '중장기/...","['자산/NNG;건전성/NNG;강화/NNG', '실적/NNG;악화/NNG', '실적...",0
73975,2013-01-02,[단기자금시장 분석] 통안채 만기..지준 잉여,2일 단기자금시장은 통안채 만기 등으로 지급준비금 잉여 분위기가 이어질 것으로 예상...,"['단기자금시장/NNG', '통안채/NNG', '만기/NNG', '지준/NNG', ...","['시중/NNG;자금/NNG;많/VA;의미/NNG;반대/NNG', '자금/NNG;시...",0
73976,2013-01-02,[한창헌의 채권분석] 재정절벽 타결에 되돌림 압력,2일 서울채권시장은 미국의 재정절벽 협상이 연말 극적으로타결된 데 따라 위험자산 선...,"['채권시장/NNG', '재정절벽/NNG', '협상/NNG', '극적/VAX', '...","['외국인/NNG;국채선물/NNG;계약/NNG;넘/VV', '위험자산/NNG;선호/...",0


## **Call_Rate**

In [43]:
call_rate = pd.read_csv("call_rate.csv", header=None)
call_rate.columns = ['date','call_rate']
call_rate

,date,call_rate
0,2020-07-24,0.48
1,2020-07-23,0.46
2,2020-07-22,0.45
3,2020-07-21,0.46
4,2020-07-20,0.44
...,...,...
3915,2004-10-18,3.51
3916,2004-10-15,3.52
3917,2004-10-14,3.51
3918,2004-10-13,3.52


In [44]:
dt_index = pd.date_range(start='20041201', end='20171231')
dt_list = dt_index.strftime("%Y-%m-%d").tolist()

mk_call_rate = pd.DataFrame(dt_list ,columns = ['date'])
# mk_call_rate['call_rate'] = 0.0
mk_call_rate

,date
0,2004-12-01
1,2004-12-02
2,2004-12-03
3,2004-12-04
4,2004-12-05
...,...
4774,2017-12-27
4775,2017-12-28
4776,2017-12-29
4777,2017-12-30


In [45]:
mk_call_rate = pd.merge(mk_call_rate,call_rate, on='date')

for i in range(len(mk_call_rate)):
    if mk_call_rate['call_rate'][i] == 0:
        mk_call_rate['call_rate'][i] = mk_call_rate['call_rate'][i-1]

mk_call_rate

,date,call_rate
0,2004-12-01,3.26
1,2004-12-02,3.26
2,2004-12-03,3.26
3,2004-12-06,3.23
4,2004-12-07,3.19
...,...,...
3247,2017-12-22,1.51
3248,2017-12-26,1.52
3249,2017-12-27,1.51
3250,2017-12-28,1.55


In [46]:
mk_call_rate.to_csv('mk_call_rate.csv', index=False)

In [47]:
mk_call_rate['call_label'] = 0

for i in range(31, len(mk_call_rate)):
    if mk_call_rate['call_rate'][i] > mk_call_rate['call_rate'][i-30]:
        mk_call_rate['call_label'][i] = 'increase'
    elif mk_call_rate['call_rate'][i] < mk_call_rate['call_rate'][i-30]:
        mk_call_rate['call_label'][i] = 'decrease'
    else:
        mk_call_rate['call_label'][i] = 0

mk_call_rate

c:\python\python36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
c:\python\python36\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
c:\python\python36\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
c:\python\python36\lib\site-packages\

,date,call_rate,call_label
0,2004-12-01,3.26,0
1,2004-12-02,3.26,0
2,2004-12-03,3.26,0
3,2004-12-06,3.23,0
4,2004-12-07,3.19,0
...,...,...,...
3247,2017-12-22,1.51,increase
3248,2017-12-26,1.52,increase
3249,2017-12-27,1.51,increase
3250,2017-12-28,1.55,increase


In [48]:
mk_call_rate.to_csv('mk_call_rate_label.csv', index=False)

## **Call_Rate_labeling**

In [49]:
mk_call_rate_label = pd.read_csv("mk_call_rate_label.csv")
mk_call_rate_label

,date,call_rate,call_label
0,2004-12-01,3.26,0
1,2004-12-02,3.26,0
2,2004-12-03,3.26,0
3,2004-12-06,3.23,0
4,2004-12-07,3.19,0
...,...,...,...
3247,2017-12-22,1.51,increase
3248,2017-12-26,1.52,increase
3249,2017-12-27,1.51,increase
3250,2017-12-28,1.55,increase


In [52]:
df_yonhap_label = pd.merge(df_yonhap_test,mk_call_rate_label, on='date')
df_yonhap_label

,date,title,content,tokens,ngrams,callLabel,call_rate,call_label
0,2005-01-17,농협 상호금융 대출금리 연 5%대 초반으로 인하,현행 연 6%대 후반인 농협 상호금융 대출금리가 올해안에 연 5%대 초반까지 단계...,"['연/NNG', '농협/NNG', '상호금융/NNG', '대출/NNG', '금리/...","['농가/NNG;소득/NNG;안정/NNG', '비용/NNG;부담/NNG;덜/VV',...",decrease,3.25,increase
1,2005-01-17,<채권> 금리 상승 지속..투자심리 취약 여전(마감),채권시장의 투자심리가 여전히 취약한 가운데 금리가 또 올랐다.\r 17일 ...,"['채권시장/NNG', '투자심리/NNG', '여전히/MAG', '취약/NNG', ...","['경기/NNG;선행/NNG;지수/NNG;상승/NNG', '단기물/NNG;수급/NN...",decrease,3.25,increase
2,2005-01-17,2010년까지 부품.소재 중핵기업 300개 육성,中企근무 공고생 입영연기-대학등록금 지원 \r소상공인 특례 보증.. 정부는 오는 2...,"['中企/NNG', '근무/NNG', '공고/NNG', '입영/NNG', '연기/N...","['은행/NNG;중소기업/NNG;대출/NNG;확대/NNG', '수입/NNG;증가/N...",decrease,3.25,increase
3,2005-01-17,<中企-부품.소재 육성안 어떤 내용 담겼나>,정부가 이번에 내놓은 중소기업 대책과 부품.소재산업 육성방안은 작년 7월 발표된 ...,"['내놓/VV', '중소기업/NNG', '대책/NNG', '부품/NNG', '소재산...","['은행/NNG;중소기업/NNG;대출/NNG;확대/NNG', '대일/NNG;무역/N...",decrease,3.25,increase
4,2005-01-17,"미국인 60% ""2기 부시 희망적""",미국인들의 60%는 조지 부시 미국 대통령의 집권 2기에 대해 희망적이라는 생각을 ...,"['미국인/NNG', '집권/NNG', '대해/VV', '희망/NNG', '생각/N...","['달리/MAG;금리/NNG;인상/NNG', '금리/NNG;인상/NNG;높/VA',...",decrease,3.25,increase
...,...,...,...,...,...,...,...,...
53129,2017-12-29,브라질 자동차 판매 5년 만에 증가세…작년보다 9%↑,올해 판매량 223만7천대 추산…2020년에 300만대 넘길 듯 김재순 통신원 =...,"['판매/NNG', '넘/VV', '판매/NNG', '증가/NNG', '나타나/VV...","['물가/NNG;안정/NNG;금리/NNG;인하/NNG', '산업/NNG;긍정적/VA...",increase,1.58,increase
53130,2017-12-29,뉴욕증시 올해 마지막 거래일 앞두고 강세…다우 사상 최고 마감,뉴욕증시의 다우존스 30 산업평균지수는 올해 거래일을 하루 남겨두고 사상 최고치를 ...,"['다우존스산업평균지수/NNG', '거래일/NNG', '남기/VV', '사상/NNG...","['ff/NNG;금리선물/NNG;시장/NNG;금리/NNG;인상/NNG', '유가/N...",increase,1.58,increase
53131,2017-12-29,"""마이너스금리 일본은행서 드디어 금리인상 검토 목소리""",미국과 유럽이 세계 금융위기 뒤 지속한 금융완화에서 벗어나려는 가운데 일본은행 내에...,"['세계/NNG', '금융위기/NNG', '뒤/NNG', '지속/NNG', '금융/...","['정책/NNG;효과/NNG;부작용/NNG', '정책/NNG;금융/NNG;완화/NN...",increase,1.58,increase
53132,2017-12-29,'대출금리 조작 사건' 외환은행 전·현직 임원 무죄 확정,"고객 몰래 대출금리 인상 혐의…법원 ""무단으로 인상했다 단정 못해"" 고객 동의 없이...","['고객/NNG', '몰래/MAG', '대출/NNG', '금리/NNG', '인상/N...","['대출/NNG;스프레드/NNG;인상/NNG', '대출/NNG;금리/NNG;인상/N...",increase,1.58,increase


In [53]:
df_yonhap_in_label = pd.merge(df_yonhap_in_test,mk_call_rate_label, on='date')
df_yonhap_in_label

,date,title,content,tokens,ngrams,callLabel,call_rate,call_label
0,2005-01-18,"채권금리, 심리회복..본격 매수세 회복은 아직",18일 오후 채권시장은 심리회복에 따른 매수세가 유입되면서 주요 기물별 금리하락폭...,"['채권시장/NNG', '심리/NNG', '회복/NNG', '따르/VV', '매수/...","['채권시장/NNG;심리/NNG;회복/NNG', '금리/NNG;하락/NNG;크/VV...",0,3.25,increase
1,2005-01-18,"美 국채가, FRB 금리인상 시사 관측 亞서 하락 전망",18일 아시아에서 스크린을 통해 거래되는 미국채 가격은 연방준비제도이사회(FRB) ...,"['스크린/NNG', '통하/VV', '거래/NNG', '되/XSV', '가격/NN...","['인상/NNG;관측/NNG;강화/NNG', '경제/NNG;회복/NNG;가속/NNG...",0,3.25,increase
2,2005-01-18,"채권금리, 내림세 출발..심리 안정 기대",18일 오전 채권시장은 올들어 이어진 금리 급등흐름이 주춤하면서 내림세로 출발했다....,"['채권시장/NNG', '들/VV', '이어지/VV', '금리/NNG', '급등/N...","['금리/NNG;급등/NNG', '심리/NNG;안정/NNG', '기대/NNG;살/VV']",0,3.25,increase
3,2005-01-18,[채권-마감] 심리회복속 금리 급락 전환..추격매수는 부담,패닉 상황을 치달았던 투자심리가 18일 안정을 되찾으면서 주요 기물별 금리가 비교...,"['패닉/NNG', '치닫/VV', '투자심리/NNG', '안정/NNG', '되찾/...","['정책/NNG;불확실성/NNG;해소/NNG', '투자심리/NNG;안정/NNG', ...",0,3.25,increase
4,2005-01-18,"채권금리,조정시점..하락폭은 크지 않을 듯",18일 오전 채권시장에서 금리가 최근의 급등양상이 일단락되면서 조정권에 진입했다....,"['채권시장/NNG', '금리/NNG', '급등/NNG', '양상/NNG', '일단...","['채권시장/NNG;금리/NNG;급등/NNG', '국채/NNG;발행/NNG;불확실성...",0,3.25,increase
...,...,...,...,...,...,...,...,...
71510,2013-01-02,[국채선물 전략] 미국發 약세 압력,2일 국채선물시장은 미국 재정절벽 타결 협상에 대한 기대 등으로 약세 압력을 받을 ...,"['국채선물시장/NNG', '재정절벽/NNG', '타결/NNG', '협상/NNG',...","['국채선물/NNG;가격/NNG;하락/NNG', '안전자산/NNG;선호/NNG;약화...",0,2.77,increase
71511,2013-01-02,"박재식 한국증금 사장 ""실적악화 불가피, 중장기 로드맵 필수""","박재식 한국증권금융 사장은 2일 ""실적 악화가 불가피한 올해엔 중장기 성장 동력을 ...","['실적/NNG', '악화/NNG', '불가피/NNG', '엔/NNG', '중장기/...","['자산/NNG;건전성/NNG;강화/NNG', '실적/NNG;악화/NNG', '실적...",0,2.77,increase
71512,2013-01-02,[단기자금시장 분석] 통안채 만기..지준 잉여,2일 단기자금시장은 통안채 만기 등으로 지급준비금 잉여 분위기가 이어질 것으로 예상...,"['단기자금시장/NNG', '통안채/NNG', '만기/NNG', '지준/NNG', ...","['시중/NNG;자금/NNG;많/VA;의미/NNG;반대/NNG', '자금/NNG;시...",0,2.77,increase
71513,2013-01-02,[한창헌의 채권분석] 재정절벽 타결에 되돌림 압력,2일 서울채권시장은 미국의 재정절벽 협상이 연말 극적으로타결된 데 따라 위험자산 선...,"['채권시장/NNG', '재정절벽/NNG', '협상/NNG', '극적/VAX', '...","['외국인/NNG;국채선물/NNG;계약/NNG;넘/VV', '위험자산/NNG;선호/...",0,2.77,increase


In [54]:
df_yonhap_label.to_csv('df_yonhap_label.csv', index=False)

In [55]:
df_yonhap_in_label.to_csv('df_yonhap_in_label.csv', index=False)

## **call_rate_test**

In [13]:
call_rate['date'][3883]

'2004-12-01'

In [34]:
df_call['date'][100]

'2005-04-11'

In [ ]:
df_call['date'][i] == call_rate['date'][j]

In [44]:
for i in range(100):
    for j in range(3800, 3860):
        if df_call['date'][i] == call_rate['date'][j]:
            df_call['call_late'][i] = call_rate['call_late'][j]
df_call

c:\users\ehhah\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,date,call_late
0,2005-01-01,0.00
1,2005-01-02,0.00
2,2005-01-03,0.00
3,2005-01-04,3.27
4,2005-01-05,3.28
...,...,...
4743,2017-12-27,0.00
4744,2017-12-28,0.00
4745,2017-12-29,0.00
4746,2017-12-30,0.00
